# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
# ! pip install -q schedule pytest # установка библиотек, если ещё не

In [1]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import re
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [2]:
def get_book_data(book_url: str) -> dict:
    """
    Возвращает словарь с данными об одной книге.
    
    Args:
        book_url (str): url-адрес страницы книги.

    Returns:
        product_info (dict): словарь с данными о книге.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, "html.parser")
    product_info = {}
    if soup.find("h1"):
        product_info['Title'] = soup.find("h1").get_text()
    else:
        product_info['Title'] = 'The title is absent'
    if soup.find("p", attrs={"class": "price_color"}):
        product_info['Price'] = soup.find(
            "p",
            attrs={"class": "price_color"}
        ).get_text()[1:]
    else:
        product_info['Price'] = 'The price is absent'    
    if soup.find("p", class_=re.compile(r"star-rating\s\w+")):
        product_info['Star-rating'] = soup.find(
            "p",
            class_=re.compile(r"star-rating\s\w+")
        )['class'][1]
    else:
        product_info['Star-rating'] = 'The star-rating is not given'
    if  soup.find("p", attrs={"class": "instock availability"}):
        product_info['Available quantity'] = re.search(
            r"\((\d{0,1000}) available\)",
            soup.find("p", attrs={"class": "instock availability"}).get_text()
        ).group(1)
    else:
        product_info['Available quantity'] = \
            'The available quantity is not given' 
    if soup.find("div", attrs={"id": "product_description"}):
        product_info['Description'] = soup.find(
            "div", 
            attrs={"id": "product_description"}
        ).find_next_sibling().get_text()
    else:
        product_info['Description'] = 'Description is absent'
    table_info = soup.find("table", attrs={"class": "table table-striped"})
    if table_info:
        for i, row in enumerate(table_info.find_all("tr")):
            if i in [2, 3, 4]:
                product_info[row.find("th").get_text()] = \
                    row.find("td").get_text()[1:]
            else:
                product_info[row.find("th").get_text()] = \
                    row.find("td").get_text()
    return product_info
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [3]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'Title': 'A Light in the Attic',
 'Price': '£51.77',
 'Star-rating': 'Three',
 'Available quantity': '22',
 'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to r

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [4]:
def scrape_books(is_save: bool = False) -> list:
    """
    Возвращает список словарей с данными о книгах.
    Сохраняет список словарей в файл.
    
    Args:
        is_save (bool): аргумент-флаг - сохранение в файл.

    Returns:
        books_data (list): список словарей с данными о книгах.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    base_url = "https://books.toscrape.com/catalogue/page-1.html"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, "html.parser")
    pages_count = int(re.search(
        r"Page\s1\sof\s(\d{2,4})",
        soup.find("li", attrs={"class": "current"}).get_text()
    ).group(1))
    books_data = []
    for i in range(1, pages_count+1):
        current_url = f"https://books.toscrape.com/catalogue/page-{i}.html"
        response = requests.get(current_url)
        soup = BeautifulSoup(response.text, "html.parser")
        books_lst = soup.find("ol", attrs={"class": "row"}).find_all("li")
        books_ref_lst = [book.find("a")["href"] for book in books_lst]
        for ref in books_ref_lst:
            book_url = 'https://books.toscrape.com/catalogue/' + ref
            books_data.append(get_book_data(book_url))
    if is_save:
        with open('../artifacts/books_data.txt', 'w', encoding='utf-8') as f:
            for book in books_data:
                f.write(str(book) + '\n')
    return books_data
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [5]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day.at("19:00").do(scrape_books(is_save=True))
while True:
    schedule.run_pending()
    time.sleep(60)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
# Проверка работоспособности функции
def scheduled_scrape_check():
    """ 
    Вспомогательная функция.
    Выводит тип и размер данных, генерируемых функцией scrape_books.
    """
    res = scrape_books(is_save=True)
    print(type(res), len(res))
    
schedule.every().day.at("20:54").do(scheduled_scrape_check)
while True:
    schedule.run_pending()
    time.sleep(60)

<class 'list'> 1000


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest ../tests/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.13.5, pytest-8.3.4, pluggy-1.5.0
rootdir: C:\Users\Роман\HW_03_1\books_scraper
plugins: anyio-4.7.0
collected 3 items

..\tests\test_scraper.py ...                                             [100%]

======================== 3 passed in 475.76s (0:07:55) ========================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```